In [3]:
# !pip install pulp numpy pandas

In [2]:
import pulp
import numpy as np
import pandas as pd
from pulp import *

In [5]:
# Create simulated dataset
np.random.seed(42)
data = {
    'Channel': ['TV', 'Radio', 'Digital', 'Print', 'Outdoor'],
    'Min_Budget': [10000, 5000, 8000, 3000, 2000],
    'Max_Budget': [50000, 20000, 40000, 15000, 10000],
    'ROI': [2.5, 1.8, 3.2, 1.5, 1.2],  # Sales generated per $1 spent
    'Impact_Score': [0.9, 0.6, 0.8, 0.5, 0.4]  # Brand impact metric
}

In [6]:
df = pd.DataFrame(data)
total_budget = 100000

In [7]:
# Create the optimization problem
prob = LpProblem("Marketing_Mix_Optimization", LpMaximize)

In [8]:
# Create decision variables
channels = list(df['Channel'])
x = LpVariable.dicts("Budget", channels, lowBound=0)

In [9]:
# Objective function: Maximize total sales
prob += lpSum([df.loc[df['Channel']==c, 'ROI'].values[0] * x[c] for c in channels])

In [10]:
# Constraints
# 1. Total budget constraint
prob += lpSum([x[c] for c in channels]) <= total_budget

In [11]:
# 2. Channel-specific constraints
for c in channels:
    min_b = df.loc[df['Channel']==c, 'Min_Budget'].values[0]
    max_b = df.loc[df['Channel']==c, 'Max_Budget'].values[0]
    prob += x[c] >= min_b
    prob += x[c] <= max_b

In [12]:
# 3. Brand impact constraint (weighted average >= 0.7)
prob += lpSum([df.loc[df['Channel']==c, 'Impact_Score'].values[0] * x[c] for c in channels]) / total_budget >= 0.7

In [13]:
# Solve the problem
prob.solve()

1

In [14]:
# Display results
print("Status:", LpStatus[prob.status])
print("\nOptimal Budget Allocation:")
for c in channels:
    print(f"{c}: ${value(x[c]):,.2f}")

print(f"\nTotal Budget Allocated: ${sum(value(x[c]) for c in channels):,.2f}")
print(f"Projected Sales: ${value(prob.objective):,.2f}")

# Calculate ROI
print(f"\nROI Analysis:")
for c in channels:
    spend = value(x[c])
    roi = df.loc[df['Channel']==c, 'ROI'].values[0]
    print(f"{c}: Every $1 → ${roi:.2f} sales (Total: ${spend*roi:,.2f})")

# Shadow price analysis
print("\nConstraint Sensitivity (Shadow Prices):")
for name, constraint in prob.constraints.items():
    print(f"{name}: {constraint.pi:.4f} (Slack: {constraint.slack:.2f})")

Status: Optimal

Optimal Budget Allocation:
TV: $50,000.00
Radio: $5,000.00
Digital: $40,000.00
Print: $3,000.00
Outdoor: $2,000.00

Total Budget Allocated: $100,000.00
Projected Sales: $268,900.00

ROI Analysis:
TV: Every $1 → $2.50 sales (Total: $125,000.00)
Radio: Every $1 → $1.80 sales (Total: $9,000.00)
Digital: Every $1 → $3.20 sales (Total: $128,000.00)
Print: Every $1 → $1.50 sales (Total: $4,500.00)
Outdoor: Every $1 → $1.20 sales (Total: $2,400.00)

Constraint Sensitivity (Shadow Prices):
_C1: 1.8000 (Slack: -0.00)
_C2: -0.0000 (Slack: -40000.00)
_C3: 0.7000 (Slack: -0.00)
_C4: -0.0000 (Slack: -0.00)
_C5: -0.0000 (Slack: 15000.00)
_C6: -0.0000 (Slack: -32000.00)
_C7: 1.4000 (Slack: -0.00)
_C8: -0.3000 (Slack: -0.00)
_C9: -0.0000 (Slack: 12000.00)
_C10: -0.6000 (Slack: -0.00)
_C11: -0.0000 (Slack: 8000.00)
_C12: -0.0000 (Slack: -0.12)
